In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 450kB 33.4MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
import pickle
from konlpy.tag import Okt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
pd.set_option('display.unicode.east_asian_width', True)

In [ ]:
with open('/content/datasets/tbt_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [ ]:
with open('/content/datasets/category_encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)
category = encoder.classes_
print(category)

['Baby,Kids' 'Beauty' 'Deco' 'FasihonClothing' 'FasionGoods' 'Food'
 'Furniture' 'Jewelry' 'Kitchen' 'Life']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
df_news_today = pd.read_csv(
    '/content/datasets/test_data_coupang.csv',
    index_col=0)
print(df_news_today.head())

                                                      title   category
0                      코멧  단 라탄 서랍장  화이트          Furniture
1             코멧 코팅 논슬립 옷걸이  그레이    개          Furniture
2  리빙톡 조립식 신발정리함       매직행주      세트 ......  Furniture
3              고리회전 논슬립 옷걸이  화이트    개          Furniture
4    스칸디나 멀티테이블      티크판넬   블랙프레임          Furniture


In [ ]:
print(df_news_today.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     600 non-null    object
 1   category  600 non-null    object
dtypes: object(2)
memory usage: 14.1+ KB
None


In [ ]:
df = df_news_today.drop_duplicates(subset=['title'])
sum_dup = df.title.duplicated().sum()
print(sum_dup)

0


In [ ]:
df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     574 non-null    object
 1   category  574 non-null    object
dtypes: object(2)
memory usage: 13.5+ KB


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
X = df['title']
Y = df['category']

In [ ]:
print(X[1])

코멧 코팅 논슬립 옷걸이  그레이    개 


In [ ]:
okt = Okt()
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0                   [코멧, 단, 라, 탄, 서랍, 장, 화이트]      
1           [코멧, 코팅, 논, 슬립, 옷, 걸이, 그레이, 개]      
2      [리빙톡, 조립, 식, 신발, 정리, 함, 매직, 행주, 세트]...
3           [고리, 회전, 논, 슬립, 옷, 걸이, 화이트, 개]      
4      [스칸, 디나, 멀티, 테이블, 티크, 판넬, 블랙, 프레임]...
                                ...                           
569    [온리히어, 천연, 원, 석, 종, 건강, 팔찌, 선물, 용, 케이스, 인증서]...
570         [알, 뜨리, 귀걸이, 보관, 함, 단, 투명, 개입]      
571    [황, 보석, 순은, 실버, 팔찌, 은, 팔찌, 체인, 팔찌, 팔찌]...
572    [비즈, 도구, 비즈, 주얼리, 공구, 세트, 종, 집게, 절단, 니, 퍼, 구자,...
573    [제이, 에스, 글로벌, 핑크, 돼지, 요술, 허리, 핏, 단추, 단추]...
Name: title, Length: 574, dtype: object


In [ ]:
stopwords = pd.read_csv('/content/datasets/stopwords2.csv')

In [ ]:
for i in range(len(X)):
    result = []
    for j in range(len(X[i])):
        if len(X[i][j]) > 1:
            if X[i][j] not in list(stopwords['stopword']):
                result.append(X[i][j])
    X[i] = ' '.join(result)
print(X)            

0                                              코멧 서랍
1                             코멧 코팅 슬립 걸이 그레이
2                        리빙톡 조립 신발 정리 매직 행주
3                                    고리 회전 슬립 걸이
4                 스칸 디나 멀티 테이블 티크 판넬 프레임
                             ...                        
569                온리히어 천연 건강 팔찌 케이스 인증서
570                                뜨리 귀걸이 보관 투명
571                   보석 순은 팔찌 팔찌 체인 팔찌 팔찌
572    비즈 도구 비즈 주얼리 공구 집게 절단 구자 말이집 오링 반지 바늘 송곳 도구...
573            제이 에스 글로벌 돼지 요술 허리 단추 단추
Name: title, Length: 574, dtype: object


In [ ]:
tokened_X = token.texts_to_sequences(X)
print(tokened_X)

[[82], [950, 211, 18, 88], [771, 449, 31, 377, 297], [437, 526, 211, 18], [1722, 4128, 157, 8, 583, 224], [5042, 1135, 9468, 77, 242, 1162, 14387], [483, 54], [211, 950, 526, 315, 171, 18], [8010, 571, 22, 1819, 1079, 87, 218, 703, 87, 902, 1819, 1079, 87, 136, 79, 136], [841, 243], [8398, 159, 282, 291, 18], [36, 332, 302, 2011, 535], [211, 171, 18, 88], [885, 48, 43, 663, 8], [211, 18, 88], [7057, 638, 997, 282, 18, 377, 291, 819], [211, 950, 171, 18, 88], [7278, 43, 106, 291, 53], [153, 54], [4294, 2247, 241, 168, 302], [130, 5755, 771, 1316, 904], [6604, 308, 224, 54, 2011, 535], [211, 526, 18, 88], [82, 1064, 88], [422, 516, 1162, 88], [236, 1722, 4128, 8, 224], [448, 5755, 1848, 20, 705], [68, 168, 302], [157, 243], [236, 1722, 4128, 157, 8, 583, 224, 9709], [4247, 43, 31], [1545, 792, 291, 14, 98, 6, 291], [885, 30, 140, 1087, 1162, 142], [153, 54], [2420, 4848, 474, 596, 8], [1532, 20, 244, 18, 302, 4555, 9709], [3079, 377, 448, 2273, 24, 60, 2066, 4872, 215, 88], [12179, 1332,

In [ ]:
#max 값 확인 
max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):
        max = len(tokened_X[i])
print(max)

18


In [ ]:
X_pad = pad_sequences(tokened_X, 13)
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
     82]
 [    0     0     0     0     0     0     0     0     0   950   211    18
     88]
 [    0     0     0     0     0     0     0     0   771   449    31   377
    297]
 [    0     0     0     0     0     0     0     0     0   437   526   211
     18]
 [    0     0     0     0     0     0     0  1722  4128   157     8   583
    224]
 [    0     0     0     0     0     0  5042  1135  9468    77   242  1162
  14387]
 [    0     0     0     0     0     0     0     0     0     0     0   483
     54]
 [    0     0     0     0     0     0     0   211   950   526   315   171
     18]
 [ 1819  1079    87   218   703    87   902  1819  1079    87   136    79
    136]
 [    0     0     0     0     0     0     0     0     0     0     0   841
    243]]


In [ ]:
model = load_model('/content/datasets/10x10_classfication_0.9141626358032227.h5')

In [ ]:
predict = model.predict(X_pad)
print(predict[0])

[5.2269224e-05 3.1469151e-04 2.6011716e-03 7.9198537e-07 2.6086043e-05
 5.9336294e-06 9.9466360e-01 1.6714512e-04 1.0582949e-03 1.1100427e-03]


In [ ]:
result = np.argmax(predict[0])
print(category[result])

Furniture


In [ ]:
predict_category = []
for pred in predict:
    predict_category.append(category[np.argmax(pred)])
print(predict_category)

['Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Kitchen', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Beauty', 'Life', 'Life', 'Beauty', 'Life', 'Life', 'Life', 'Kitchen', 'Life', 'Life', 'Life', 'Life', 'Life', 'Life', 'Life', 'Beauty', 'Baby,Kids', 'Beauty', 'Life', 'Life', 'Life', 'Life', 'Deco', 'Life', 'Beauty', 'Life', 'Life', 'Life', 'Life', 'Life', 'Kitchen', 'Life', 'Life', 

In [ ]:
print(Y)

0      Furniture
1      Furniture
2      Furniture
3      Furniture
4      Furniture
         ...    
569      Jewelry
570      Jewelry
571      Jewelry
572      Jewelry
573      Jewelry
Name: category, Length: 574, dtype: object


In [ ]:
df['predict'] = predict_category
print(df.head())

                                    title   category    predict
0                               코멧 서랍  Furniture  Furniture
1              코멧 코팅 슬립 걸이 그레이  Furniture  Furniture
2         리빙톡 조립 신발 정리 매직 행주  Furniture       Life
3                     고리 회전 슬립 걸이  Furniture  Furniture
4  스칸 디나 멀티 테이블 티크 판넬 프레임  Furniture  Furniture


In [ ]:
df['OX'] = 0
print(df.head())

                                    title   category    predict  OX
0                               코멧 서랍  Furniture  Furniture   0
1              코멧 코팅 슬립 걸이 그레이  Furniture  Furniture   0
2         리빙톡 조립 신발 정리 매직 행주  Furniture       Life   0
3                     고리 회전 슬립 걸이  Furniture  Furniture   0
4  스칸 디나 멀티 테이블 티크 판넬 프레임  Furniture  Furniture   0


In [ ]:
for i in range(len(df.predict)):
    if df.category[i] == df.predict[i]:
        df.OX[i] = 'O'
    else:
         df.OX[i] = 'X'
print(df.head())

                                    title   category    predict OX
0                               코멧 서랍  Furniture  Furniture  O
1              코멧 코팅 슬립 걸이 그레이  Furniture  Furniture  O
2         리빙톡 조립 신발 정리 매직 행주  Furniture       Life  X
3                     고리 회전 슬립 걸이  Furniture  Furniture  O
4  스칸 디나 멀티 테이블 티크 판넬 프레임  Furniture  Furniture  O


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
print(df.OX.value_counts() / len(df.OX))

O    0.665505
X    0.334495
Name: OX, dtype: float64


In [ ]:
loc_list = []
for i in range(len(df.predict)):
    if df.category[i] != df.predict[i]:
      loc_list.append(i)

print(len(loc_list))
#print(df.loc[loc_list][:100])
df.loc[loc_list][-100 : ]

192


,title,category,predict,OX
317,앱솔 루트 킨더밀쉬 우유,"Baby,Kids",FasionGoods,X
319,다우니 엑스 퍼트 센서티브 섬유 유연제 베이비 파우더...,"Baby,Kids",Life,X
320,비책 일드 클렌징 티슈,"Baby,Kids",Beauty,X
322,탐사 학습 교재 용량 믹스 색종이,"Baby,Kids",Deco,X
323,고려 은단 비타민,"Baby,Kids",Food,X
...,...,...,...,...
564,케어 프린 티타늄 정전기 완화 팔찌,Jewelry,Life,X
565,한국 동산 감정 천사 날개 순금 반지 백일 첫돌,Jewelry,Furniture,X
567,푸디 크리스탈 악세사리 유리 보석,Jewelry,Furniture,X
569,온리히어 천연 건강 팔찌 케이스 인증서,Jewelry,Furniture,X


In [ ]:
df.iloc[-100: , 0:3]

,title,category,predict
474,부케 가르니 퍼퓸 비누 베이비 파우더,Beauty,Beauty
475,닥터 방기 나노 버블 클렌징폼,Beauty,Beauty
476,이니스프리 그린티 고수 씨드,Beauty,Beauty
477,에스쁘 프로 테일러 실크 쿠션 파운데이션 리필 아이보리...,Beauty,Beauty
478,미샤 타임 레볼루션 개똥쑥 트리트먼트 에센스,Beauty,Beauty
...,...,...,...
569,온리히어 천연 건강 팔찌 케이스 인증서,Jewelry,Furniture
570,뜨리 귀걸이 보관 투명,Jewelry,Jewelry
571,보석 순은 팔찌 팔찌 체인 팔찌 팔찌,Jewelry,Jewelry
572,비즈 도구 비즈 주얼리 공구 집게 절단 구자 말이집 오링 반지 바늘 송곳 도구...,Jewelry,Jewelry
